In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['KAGGLE_USERNAME']="daominhkhanh"
os.environ['KAGGLE_KEY']="90e6f266906e9dcfeae1afea9a0630ac"
!kaggle datasets download -d adityajn105/flickr8k

In [ ]:
!unzip flickr8k.zip
!mkdir Flickr8k
!mv Images Flickr8k
!rm caption.txt 

In [ ]:
!rm flickr8k.zip

In [ ]:
!kaggle datasets download -d authman/pickled-glove840b300d-for-10sec-loading

In [ ]:
!unzip pickled-glove840b300d-for-10sec-loading.zip

In [2]:
import torch
from torch import optim
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader,random_split,Dataset
from torchvision import transforms as T
import pickle
from torch import nn
import numpy as np 
from PIL import Image 
import pandas as pd 
import time
import pandas as pd
import os
import random
import numpy as np
from string import punctuation
from torchvision import transforms as T
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.text import Tokenizer
import re

In [ ]:
!cp /content/drive//MyDrive/Attention2/captions.txt /content/Flickr8k/

In [ ]:
embedding_dict=np.load('glove.840B.300d.pkl',allow_pickle=True)

In [3]:
contractions_dict = { "ain’t": "are not", "’s":" is", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "‘cause": "because", "could’ve": "could have", "couldn’t": "could not", "couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have", "hasn’t": "has not", "haven’t": "have not", "he’d": "he would", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "I’d": "I would", "I’d’ve": "I would have", "I’ll": "I will", "I’ll’ve": "I will have", "I’m": "I am", "I’ve": "I have", "isn’t": "is not", "it’d": "it would", "it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "let’s": "let us", "ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not", "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will", "she’ll’ve": "she will have", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have", "so’ve": "so have", "that’d": "that would", "that’d’ve": "that would have", "there’d": "there would", "there’d’ve": "there would have", "they’d": "they would", "they’d’ve": "they would have","they’ll": "they will",
 "they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would", "we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not","what’ll": "what will", "what’ll’ve": "what will have", "what’re": "what are", "what’ve": "what have", "when’ve": "when have", "where’d": "where did", "where’ve": "where have",
 "who’ll": "who will", "who’ll’ve": "who will have", "who’ve": "who have", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have", "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have", "y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have", "you’re": "you are", "you’ve": "you have"}
contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, s)


In [ ]:
captions_length=[]
data=pd.read_csv('/content/Flickr8k/captions.txt',sep=',',skipinitialspace=True)
data.caption=data.caption.apply(lambda x:expand_contractions(x,contractions_dict))
table=str.maketrans('','',punctuation)
data.caption=data.caption.apply(lambda x:x.translate(table))
data.caption=data.caption.apply(lambda x:re.sub("\s\s+"," ",x.lower()))
for caption in data.caption:
    captions_length.append(len(caption.split(' ')))

data['length']=captions_length
n_images=len(set(data.image))
images=set(data.image)


In [ ]:
print(f"We have {n_images} images")

In [ ]:
random.seed(42)
images_train=set(random.sample(images,k=6500))
images_test=images-images_train
images_train=list(images_train)
images_test=list(images_test)

In [ ]:
data_train=data[data.image.isin(images_train)]
data_test=data[data.image.isin(images_test)]

In [ ]:
def preprocessing(dataframe):
    images_deleted=set(dataframe[dataframe.length<6].image)
    dataframe=dataframe[~dataframe.image.isin(images_deleted)]
    dataframe=dataframe.sample(frac=1)
    dataframe.drop('length',axis=1,inplace=True)
    return dataframe
    
data_train=preprocessing(data_train)
data_test=preprocessing(data_test)

In [ ]:
print(f"We have {len(set(data_train.image))} images train")
print(f"We have {len(set(data_test.image))} images test")

In [ ]:
data_train.to_csv('Flickr8k/captions_train.csv',sep='|',index=False)
data_test.to_csv('Flickr8k/captions_test.csv',sep='|',index=False)

In [ ]:
!cp /content/Flickr8k/captions_train.csv /content/drive/MyDrive/Attention2

In [ ]:
!cp /content/Flickr8k/captions_test.csv /content/drive/MyDrive/Attention2/

In [ ]:
vocab_size=6000

In [ ]:
class Vocabulary:
    def __init__(self):
        self.vocab_size=6000
        self.tokenizer=Tokenizer(num_words=self.vocab_size,
                                filters='!"#$%&()*+/:;"=?@[\\]^_`{|}~\t\n',
                                oov_token='<unk>'
        )

    def __len__(self):
        return len(self.vocab_size)

    def build_vocab(self,captions_list):
        self.tokenizer.fit_on_texts(captions_list)
        self.sequences=self.tokenizer.texts_to_sequences(captions_list)
        self.tokenizer.word_index['<pad>']=0
        self.tokenizer.index_word[0]='<pad>'   
        self.stoi=self.tokenizer.word_index
        self.itos=self.tokenizer.index_word

    def numericalize(self,text):
        return self.tokenizer.texts_to_sequences([text])[0]


class Flickr8k(Dataset):
    def __init__(self,root_dir,caption_file,transforms=None):
        path=os.getcwd()
        self.transforms=transforms
        path_to_caption_file=os.path.join(path,root_dir+"/"+caption_file)
        df=pd.read_csv(path_to_caption_file,sep='|',skipinitialspace=True)
        self.images=df.image
        self.captions='<sos> '+df.caption+' <eos>'
        self.captions=self.captions.tolist()
        self.vocabulary=Vocabulary()
        self.vocabulary.build_vocab(self.captions)

    def __len__(self):
        return len(self.images)

    def __getitem__(self,idx):
        image_name=self.images[idx]
        path=os.path.join(os.getcwd(),"Flickr8k/Images/"+str(image_name))
        image=Image.open(path).convert("RGB")
        if self.transforms is not None:
            image=self.transforms(image)
        caption_vector=self.vocabulary.sequences[idx]
        return image,torch.tensor(caption_vector)



class MyCollate:
    def __init__(self,pad_idx):
        self.pad_idx=pad_idx

    def __call__(self,batch):
        #batch has a lot of pair (image,caption)      
        images=[item[0].unsqueeze(dim=0) for item in batch]
        captions=[item[1] for item in batch]
        images=torch.cat(images,dim=0)#batch_size*3*height*width
        captions=pad_sequence(captions,batch_first=True,padding_value=self.pad_idx)
        return images,captions


def get_dataset(root_dir,caption_file,transforms,batch_size=64,num_workers=2,shuffle=False,pin_memory=True):
    dataset=Flickr8k(root_dir=root_dir,
                    caption_file=caption_file,
                    transforms=transforms,
    )
    pad_idx=dataset.vocabulary.stoi['<pad>']

    data_loader=DataLoader(dataset=dataset,
                            batch_size=batch_size,
                            num_workers=num_workers,
                            shuffle=shuffle,
                            pin_memory=pin_memory,#if pin_memory=True, which enables fast data transfer to CUDA-enable GPUS
                            collate_fn=MyCollate(pad_idx)
    )
    return dataset,data_loader



In [ ]:
transforms=T.Compose([
        T.Resize(226),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225))
    ])
dataset,data_loader=get_dataset(
            root_dir="Flickr8k",
            caption_file="captions_train.csv",
            transforms=transforms
    )

In [ ]:
# from sklearn.model_selection import train_test_split
# train_idx,val_idx=train_test_split(np.arange(len(data_loader)),test_size=0.2,random_state=42)

In [ ]:
with open('/content/drive/MyDrive/Attention2/dataset.pickle','wb') as file:
    pickle.dump(dataset,file,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
test=pd.read_csv('Flickr8k/captions_test.csv',sep='|',skipinitialspace=True)

In [ ]:
print(len(dataset.vocabulary))

In [ ]:
beam_size=3

In [ ]:
import torch
from torch import nn
from torchvision import models
from torch.nn import functional
from torch.autograd import Variable
import numpy as np 
import numpy as np
from collections import Counter
from torch.nn import functional as F
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Encoder(nn.Module):
    def __init__(self,units=512,encode_image_size=8):
        super(Encoder,self).__init__()
        self.resnet=models.resnet101(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad=False

        modules=list(self.resnet.children())[:-2]
        self.resnet=nn.Sequential(*modules)
        self.adap=nn.AdaptiveAvgPool2d((encode_image_size,encode_image_size))

    def forward(self,images):
        features=self.resnet(images)
        features=self.adap(features)
        features=features.permute(0,2,3,1)
        features=features.view(features.size(0),-1,features.size(-1))#batch_size*num_pixels*encoder_dim
        return features

class AttentionBahdanau(nn.Module):
    def __init__(self,attention_dim,encoder_dim,decoder_dim):
        super(AttentionBahdanau,self).__init__()
        self.attention_dim=attention_dim
        self.encoder_dim=encoder_dim
        self.decoder_dim=decoder_dim

        self.weight_decoder=nn.Linear(decoder_dim,attention_dim)
        self.weight_encoder=nn.Linear(encoder_dim,attention_dim)
        self.full=nn.Linear(attention_dim,1)
        self.relu=nn.ReLU()
        self.softmax=nn.Softmax(dim=1)

    def forward(self,encoder_out,hidden_decoder):
        #encoder_out: batch_size*num_pixel*encoder_dim
        #hidden_decoder: batch_size*decoder_dim

        energies=self.full(
                torch.tanh(
                    self.weight_encoder(encoder_out)+self.weight_decoder(hidden_decoder).unsqueeze(1)
                    )
            )#batch_size*num_pixels*1
        energies=energies.squeeze(dim=2)
        attention_weights=self.softmax(energies)#batch_size*num_pixels
        context_vector=(encoder_out*attention_weights.unsqueeze(dim=2)).sum(dim=1)#batch_size*encoder_dim
        return context_vector,attention_weights


class Decoder(nn.Module):
    def __init__(self,attention_dim,decoder_dim,embedding_dim,vocab_size,vocabulary,encoder_dim=2048,drop_pro=0.2):
        super(Decoder,self).__init__()
        self.attention_dim=attention_dim
        self.encoder_dim=encoder_dim
        self.decoder_dim=decoder_dim
        self.embedding_dim=embedding_dim
        self.vocab_size=vocab_size
        self.vocabulary=vocabulary
        self.beam_size=3
        self.embedding=nn.Embedding(vocab_size,embedding_dim)
        self.attention=AttentionBahdanau(attention_dim,encoder_dim,decoder_dim)
        self.lstm_cell1=nn.LSTMCell(input_size=embedding_dim+encoder_dim,
                        hidden_size=decoder_dim,
                        bias=True
        )   

        self.fcn=nn.Linear(decoder_dim,vocab_size)
        self.dropout=nn.Dropout(drop_pro)
        self.init_h=nn.Linear(encoder_dim,decoder_dim)
        self.init_c=nn.Linear(encoder_dim,decoder_dim)
        self.softmax=nn.Softmax(dim=1)
        self.init_weights(pretrain_embedding=True)
  
    
    def fine_tune_embeddings(self,fine_tune=True):
        for param in self.embedding.parameters():
            param.requires_grad=fine_tune
    
    def load_pretrained_embedding(self):        
        embedding_matrix=np.zeros((self.vocab_size+1,self.embedding_dim))
        for word,idx in self.vocabulary.stoi.items():
            if idx >vocab_size:
              break
            embedding_vector=embedding_dict.get(word)
            if embedding_vector is not None:
                embedding_matrix[idx]=embedding_vector
            elif word in ['<pad>','<sos>','<eos>']:
                embedding_matrix[idx]=np.random.rand(1,300)
            else:
              embedding_matrix[idx]=embedding_dict['<unk>']

        return embedding_matrix

    def init_weights(self,pretrain_embedding=False):
        if pretrain_embedding is False:
          self.embedding.weight.data.uniform_(-0.1, 0.1)
        else:
          self.load_embedding_pretrained()
          self.fine_tune_embeddings(fine_tune=False)
        self.fcn.bias.data.fill_(0)
        self.fcn.weight.data.uniform_(-0.1,0.1)

    def load_embedding_pretrained(self):
        embeddings=self.load_pretrained_embedding()
        embeddings=torch.from_numpy(embeddings.astype(np.float32)).to(device)
        self.embedding.weight=nn.Parameter(embeddings)
    
    def fine_tune_embeddings(self,fine_tune=True):
        for param in self.embedding.parameters():
            param.requires_grad=fine_tune

    def init_hidden_state(self,features):
        mean_features=features.mean(dim=1)
        h1=self.init_h(mean_features)#batch_size*decoder_dim
        c1=self.init_c(mean_features)
        return h1,c1

    def forward(self,features,captions):
        #features: batch_size*num_pixels*encoder_dim
        #captions: batch_size*seq_length
        embedded=self.embedding(captions)#batch_size*seq_length*embedding_size
        seq_length=captions.size(1)-1
        batch_size=captions.size(0)
        h1,c1=self.init_hidden_state(features)
        num_pixels=features.size(1)

        preds=torch.zeros(batch_size,seq_length,self.vocab_size).to(device)
        attention_weights=torch.zeros(batch_size,seq_length,num_pixels).to(device)

        for i in range(seq_length):
            #context_vectore:batch_size*encoder_dim
            #attention_weight:batch_size*num_pixel
            context_vector,attention_weight=self.attention(features,h1)
            attention_weights[:,i,:]=attention_weight
            lstm_input=torch.cat((embedded[:,i,:],context_vector),dim=1)
            h1,c1=self.lstm_cell1(lstm_input,(h1,c1))#batch_size*decoder_dim
            outputs=self.fcn(self.dropout(h1))#batch_size*vocab_size
            preds[:,i,:]=outputs
        return preds,attention_weights


    def image_captions(self,feature,vocabulary,max_length=30):
        start_word=torch.tensor(vocabulary.stoi['<sos>']).view(1,-1).to(device)
        embedding=self.embedding(start_word)#1*batch_size*embedding_size
        h1,c1=self.init_hidden_state(feature)
        attention_weights=[]
        caption=[]
        for _ in range(max_length):
            context,attention_weight=self.attention(feature,h1)
            attention_weights.append(attention_weight)
            lstm_in=torch.cat((embedding[:,0,:],context),axis=1)
            h1,c1=self.lstm_cell1(lstm_in,(h1,c1))#batch_size*decoder_dim
            output=self.fcn(h1)
            predict_word=output.argmax(dim=1)
            if vocabulary.itos[predict_word.item()]=="<eos>":
                break
            
            caption.append(predict_word.item())
            embedding=self.embedding(predict_word.unsqueeze(dim=0))

        return [vocabulary.itos[index] for index in caption]



    def through_model(self,feature,word,hidden_state,beam_size):
        h1,c1=hidden_state
        word=torch.tensor(word).view(1,-1).to(device)
        embedding=self.embedding(word)
        context_vector,_=self.attention(feature,h1)
        lstm_in=torch.cat((embedding[:,0,:],context_vector),dim=1)
        h1,c1=self.lstm_cell1(lstm_in,(h1,c1))
        #output self.softmax() 1*vocab_size
        output=self.softmax(self.fcn(h1)).cpu().detach().numpy()[0]
        preds=np.argsort(output)[-beam_size:]
        return preds,[h1,c1],output

    def image_caption_with_beam_search(self,feature,vocabulary,beam_size,max_length=30,max_collection=15):
        hidden_state=self.init_hidden_state(feature)
        preds,hidden_state,output=self.through_model(feature,vocabulary.stoi['<sos>'],hidden_state,beam_size)
        words=[]
        cap_collection=[]
        prob_likelihood=list()
        has_full=False
        for i in range(beam_size):
            h1,c1=hidden_state
            words.append([[h1,c1],[[preds[i]],output[i]]])
        
        for _ in range(max_length-1):
            temp=[]
            for idx,[hidden_state_curr,pair] in enumerate(words):
                last_word=pair[0][-1]
                preds,hidden_state_curr,output=self.through_model(feature,last_word,hidden_state_curr,beam_size)

                for index in preds:
                    cap_current,prob=pair[0].copy(),pair[1]
                    prob+=np.log(output[index])
                    if vocabulary.itos[index]=="<eos>":
                        cap_collection.append(cap_current)
                        prob_likelihood.append(prob/(len(cap_current)+1))
                        if len(cap_collection)==max_collection:
                            has_full=True
                            break
                    
                    else:
                        cap_current.append(index)
                        temp.append([hidden_state_curr,[cap_current,prob]])
                    
            
            if has_full is True or len(temp)==0:
                break

            words=temp
            words=sorted(words,key=lambda l:l[1][1],reverse=False)[-beam_size:]

        if len(cap_collection)==0:
          return []
        index=prob_likelihood.index(max(prob_likelihood))
        caption=cap_collection[index]
        temp=[vocabulary.itos[index] for index in caption]
        return temp      
    
    def beam_search(self,feature,vocabulary,max_length=40,beam_size=3,max_collection=10):
        k=beam_size
        #encoder_image_size=8
        encoder_dim=feature.size(-1)
        num_pixel=feature.size(1)

        feature=feature.expand(k,num_pixel,encoder_dim)
        #tensor store k previous word at each step, now they're just <sos>
        k_prev_words=torch.LongTensor([[vocabulary.stoi['<sos>']]] * k).to(device)#(k,1)
        
        #tensor store top k sequences
        seqs=k_prev_words
        top_k_scores=torch.zeros(k,1).to(device)
        complete_seqs=list()
        complete_seq_scores=list()

        step=1
        h1,c1=self.init_hidden_state(feature)
        #s is a number less than or equal to k, because sequences are removed from this 
        #process once they hit <end>
        size_collection=0
        while True:
            embeddings=self.embedding(k_prev_words).squeeze(1)#s*embedding_dim
            contex_vec,_=self.attention(feature,h1)
            lstm_in=torch.cat((embeddings,contex_vec),dim=1)
            h1,c1=self.lstm_cell1(lstm_in,(h1,c1))
            output=self.fcn(h1)
            scores=F.log_softmax(output,dim=1)
            scores=top_k_scores.expand_as(scores)+scores

            if step==1:
                top_k_scores,top_k_words=scores[0].topk(k,0,True,True)
            else:
                top_k_scores,top_k_words=scores.view(-1).topk(k,0,True,True)

            pre_word_indexs=top_k_words / vocab_size
            pre_word_indexs=pre_word_indexs.type(torch.long)
            next_word_indexs=top_k_words%vocab_size
            seqs=torch.cat((seqs[pre_word_indexs],next_word_indexs.unsqueeze(1)),dim=1)
            incomplete_indexs=[id for id, next_word in enumerate(next_word_indexs) if next_word != vocabulary.stoi["<eos>"]]
            complete_indexs=list(set(range(len(next_word_indexs)))-set(incomplete_indexs))

            if len(complete_indexs)>0:
                complete_seqs.extend(seqs[complete_indexs].tolist())
                complete_seq_scores.extend(top_k_scores[complete_indexs])
                size_collection+=len(complete_indexs)
                
            
            #k-=len(complete_indexs)
            if size_collection>= max_collection:
              break
            
            seqs=seqs[incomplete_indexs]
            h1 = h1[pre_word_indexs[incomplete_indexs]]
            c1 = c1[pre_word_indexs[incomplete_indexs]]

            feature=feature[pre_word_indexs[incomplete_indexs]]
            top_k_scores=top_k_scores[incomplete_indexs].unsqueeze(1)
            k_prev_words=next_word_indexs[incomplete_indexs].unsqueeze(1)

            if step>max_length:
                break
            step+=1
        if len(complete_seq_scores)==0:
          return []
        index=complete_seq_scores.index(max(complete_seq_scores))
        temp=complete_seqs[index]
        temp=temp[1:-1]
        return [vocabulary.itos[index] for index in temp]


In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=300
attention_dim=512
decoder_dim=512
embedding_dim=300
encoder=Encoder().to(device)
encoder_optim=optim.RMSprop(encoder.parameters(),lr=1e-4)
decoder=Decoder(attention_dim=attention_dim,decoder_dim=decoder_dim,embedding_dim=embedding_dim,vocab_size=vocab_size,vocabulary=dataset.vocabulary).to(device)
decoder_optim=optim.RMSprop(decoder.parameters(),lr=4e-4)
criterion=nn.CrossEntropyLoss()

In [ ]:
from random import randint
from matplotlib import pyplot as plt 
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
#from bleu import sentence_bleu,SmoothingFunction

smooth=SmoothingFunction()

def save_loss(loss_train,epoch):
    train=np.asarray(loss_train)
    np.save('/content/drive/MyDrive/Attention2/train{}.npy'.format(epoch),train)
    print("save loss done")


def preprocessing(image_name):
    image = Image.open("Flickr8k/Images/"+image_name)
    image=transforms(image)
    return image

def get_list_caption(image_name):
    indxs=np.where(test.image==image_name)[0]
    result=[]
    for index in indxs:
        s=str(test.iloc[index]['caption'])
        s=s.strip()
        result.append(s.split(' '))

    return result

def show_image(image,caption,caption_beam,epoch):
    image[0]=image[0]*0.229+0.485
    image[1]=image[1]*0.224+0.456
    image[2]=image[2]*0.225+0.406
    image=image.cpu().numpy().transpose((1,2,0))
    fig,ax=plt.subplots(1,1)
    img=ax.imshow(image,interpolation='nearest')
    if caption is not None:
        title='Greedy:  '+caption+'\nBeam size={}: '.format(beam_size)+caption_beam
        ax.set_title(title)
    plt.savefig('/content/drive/MyDrive/Attention2/image{}.png'.format(epoch))
    print("save image done")


def evaluate_bleu_score_test_set(epoch):
    bleu1,bleu2,bleu3,bleu4=0,0,0,0
    bleu1_beam,bleu2_beam,bleu3_beam,bleu4_beam=0,0,0,0
    images=set(test.image)
    encoder.eval()
    decoder.eval()
    file=open('/content/drive/MyDrive/Attention2/caption_result{}.txt'.format(epoch),'w')
    file.write(f'image,captionsn,caption_beam_size_{beam_size}\n')
    t=randint(0,len(images))
    with torch.no_grad():
        for idx,image_name in enumerate(images):
            image=preprocessing(image_name)
            image=image.to(device)
            feature=encoder(image.unsqueeze(dim=0))
            caption_greedy=decoder.image_captions(feature,dataset.vocabulary)
            caption_beam_search=decoder.beam_search(feature,dataset.vocabulary,beam_size=beam_size)
            if len(caption_beam_search)==0:
              continue
            list_caption=get_list_caption(image_name)
            caption=' '.join([word for word in caption_greedy])
            cap_beam=' '.join([word for word in caption_beam_search])
            
            file.write(f'{image_name},{caption},{cap_beam}\n')
            bleu1+=sentence_bleu(list_caption,caption_greedy,weights=(1,0,0,0))
            bleu2+=sentence_bleu(list_caption,caption_greedy,weights=(0.5,0.5,0,0))
            bleu3+=sentence_bleu(list_caption,caption_greedy,weights=(0.333,0.333,0.333,0))
            t4_1=sentence_bleu(list_caption,caption_greedy,weights=(0.25,0.25,0.25,0.25))
            bleu4+=t4_1

            bleu1_beam+=sentence_bleu(list_caption,caption_beam_search,weights=(1,0,0,0))
            bleu2_beam+=sentence_bleu(list_caption,caption_beam_search,weights=(0.5,0.5,0,0))
            bleu3_beam+=sentence_bleu(list_caption,caption_beam_search,weights=(0.333,0.333,0.333,0))
            t4_2=sentence_bleu(list_caption,caption_beam_search,weights=(0.25,0.25,0.25,0.25))
            bleu4_beam+=t4_2
            if idx==t:
              print(image_name)
              print(caption)
              print(cap_beam)
              show_image(image,caption,cap_beam,epoch)
    file.close()
    print("Greedy seach:")
    print("Bleu for 1-gram:",bleu1*1.0/(len(images)-cnt))
    print("Bleu for 2-gram:",bleu2*1.0/(len(images)-cnt))
    print("Bleu for 3-gram:",bleu3*1.0/(len(images)-cnt))
    print("Bleu for 4-gram:",bleu4*1.0/(len(images)-cnt))

    print("Beam seach:")
    print("Bleu for 1-gram:",bleu1_beam*1.0/(len(images)-cnt))
    print("Bleu for 2-gram:",bleu2_beam*1.0/(len(images)-cnt))
    print("Bleu for 3-gram:",bleu3_beam*1.0/(len(images)-cnt))
    print("Bleu for 4-gram:",bleu4_beam*1.0/(len(images)-cnt))
def save_model(epoch):
    model_state={
        'epochs':epochs,
        'encoder_state_dict':encoder.state_dict(),
        'decoder_state_dict':decoder.state_dict(),
        'encoder_optim_state_dict':encoder_optim.state_dict(),
        'decoder_optim_state_dict':encoder_optim.state_dict()
    }
    torch.save(model_state,"/content/drive/MyDrive/Attention2/model{}.pth".format(epoch))
    print("save model done")


In [ ]:
import time
loss_train=[]
for epoch in range(0,epochs+1):
    start_time=time.time()
    encoder.train()
    decoder.train()
    for idx,(images,captions) in enumerate(data_loader):
        images=images.to(device)
        captions=captions.to(device)
        features=encoder(images)
        outputs,attention_weights=decoder(features,captions)
        targets=captions[:,1:]
        loss=criterion(outputs.reshape(-1,vocab_size),targets.reshape(-1))
        encoder_optim.zero_grad()
        decoder_optim.zero_grad()
        loss.backward()
        encoder_optim.step()
        decoder_optim.step()
        if(idx%50==0):
          print(idx,end=' ')
    print()

    loss_train.append(loss.item())
    print("Epoch:{}----Train:{}----Run:{}s".format(epoch,loss.item(),-start_time+time.time()))
    if epoch!=0 and epoch %5==0:
        save_loss(loss_train,epoch)
        save_model(epoch)
        evaluate_bleu_score_test_set(epoch)  